In [ ]:
from json import loads, load, dumps, dump
from pandas import DataFrame
from urllib.parse import urlencode
from urllib.request import Request, urlopen
from lxml import html
from time import sleep
import time
import numpy as np
import html5lib
from selenium import webdriver
import lxml
from selenium.webdriver.chrome.options import Options
from functools import wraps
from copy import deepcopy
import warnings
from bs4 import BeautifulSoup
import pandas as pd
import re
warnings.filterwarnings('ignore')

In [ ]:
response_data = [
                 'categories',
                 'seriess',
                 'tags',
                 'releases',
                 'release_dates',
                 'sources',
                 'vintage_dates',
                 'observations'
                 ]

dates = [
        'realtime_start',
        'realtime_end',
        'date',
        'vintage_dates',
        'last_updated',
        'observation_start',
        'observation_end',
        'created'
        ]

In [ ]:
def get_api_key():
    available_keys = ('67d814f93096f8d3ac2637f7b784ceb3', \
                      '470c07aeaf8863bba2307d69e4c0a5d8', \
                      '3561218e0baa9fc3d088e4031341d59f', \
                      '631b29fdae2d62d9a6d1e238f3818730', 'fe59c7cb27f3a567b0c061e2f048dd96', '647de62c05056467af46b5f0feaaee39',
                                'b2179af8eca553d1632215156a1bec22',
                               'b6cde9ca344846e2bb94598605b68eab', '82a7cb2dc6157fa9b01232141116e1fb',
                               '0c5106333ca0d69697a445511ddfe77e', '77bc90a59e2c36a47bc33a98df08c640',
                               'd83b3ce2326b16df323285a422bb5058')
    return np.random.choice(available_keys)

In [ ]:
url_root = 'https://api.stlouisfed.org/fred'

In [ ]:
def _convert(frame):
    frame = frame.apply(pd.to_numeric, errors='ignore')
    for column in frame:
        if column in dates:
            frame[column] = pd.to_datetime(frame[column], utc=True)
    return frame


def _data_frame(content):
    response = loads(content)
    key = [x for x in response.keys() if x in response_data][0]
    frame = DataFrame(response[key])
    final_frame = _convert(frame)
    return final_frame


def _fetch_obs(url):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    
    driver.get(url)
    content = driver.page_source
    driver.quit()
    return content


def _url_builder(url_root, api_key, path, params):
    params['api_key'] = api_key
    url_end = urlencode(params)
    url = "%s%s%s" % (url_root,path,url_end)
    return url

def _get_request_obs(url_root,api_key,path,params):
    url = _url_builder(url_root,api_key,path,params)
    content = _fetch_obs(url)
    return content

    
def query_params(*frb_fred_params):
    def _wrapper(func):
        @wraps(func)
        def _wrapped(*args, **kwargs):
            params = kwargs.pop('params', {})
            for p in frb_fred_params:
                if p in kwargs:
                    params[p] = kwargs.pop(p)
            return func(*args,params=params,**kwargs)
        return _wrapped
    return _wrapper


@query_params('search_type','realtime_start','realtime_end',
                  'limit','offset','order_by','sort_order','filter_variable',
                  'filter_value','tag_names','exclude_tag_names')
def observations(series_id=None,params=None):
        path = '/series/observations?'
        params['series_id'] = series_id
        api_key = get_api_key()
        response = _get_request_obs(url_root,api_key,path,params)
        return response

In [ ]:
# load file with id of time series
# files are obtained using id loader notebook
with open('begin_categ_series_32263.json') as json_file:
    series_id_data = load(json_file)

In [ ]:
# distribute time series by their frequincy


dayly_ids = []
weekly_ids = []
monthly_ids = []
quarterly_ids = []
annualy_ids = []


for key, value in series_id_data.items():
    for pair in value:
        if pair[1] == 'D':
            dayly_ids.append(pair[0])
        elif pair[1] == 'W':
            weekly_ids.append(pair[0])
        elif pair[1] == 'M':
            monthly_ids.append(pair[0])
        elif pair[1] == 'Q':
            quarterly_ids.append(pair[0])
        elif pair[1] == 'A':
            annualy_ids.append(pair[0])

In [ ]:
# check how many time seriess
len(dayly_ids) + len(weekly_ids) + len(monthly_ids) + len(quarterly_ids) + len(annualy_ids)

In [ ]:
def parse_page(content):
    res_strings = re.findall('date="\d{4}-\d{2}-\d{2}" value="[-+]?\d*\.*\d+"|date="\d{4}-\d{2}-\d{2}" value="."', content)
    
    ts_values = []
    for r_s in res_strings:
        value = r_s.split('value="')[1][:-1]
        ts_values.append(value)
        
    return ts_values

In [ ]:
def append_observation(path, data):
    with open(path, 'a') as f:
        dump(data, f)

### following kernels are for loading time seriess with different frequences

### if error happen, just look at i variable and change it in the beginning of the kernel

### restart the kernel (continue loading)

In [ ]:
i = 0

path = f"fred_series_5_first_categories/dayly_first_5_categories.txt"
f = open(path, "a+")

while i < len(dayly_ids):
    dayly_id = dayly_ids[i]
    observations_page = observations(series_id=dayly_id,\
                                     realtime_start='2019-11-01', \
                                     realtime_end='2019-11-01')
    ts_values = parse_page(observations_page)
    
    f.write(','.join(ts_values) + '\n')
    
    # to see progress
    i += 1
    if i % 100 == 0:
        print(i)
        
f.close()

In [ ]:
i

In [ ]:
i = 0

path = f"fred_series_5_first_categories/weekly_first_5_categories.txt"
f = open(path, "a+")

while i < len(weekly_ids):
    weekly_id = weekly_ids[i]
    observations_page = observations(series_id=weekly_id,\
                                     realtime_start='2019-11-01', \
                                     realtime_end='2019-11-01')
    ts_values = parse_page(observations_page)
    
    f.write(','.join(ts_values) + '\n')
    
    # to see progress
    i += 1
    if i % 100 == 0:
        print(i)
        
f.close()

In [ ]:
i

In [ ]:
i = 0

path = f"fred_series_5_first_categories/monthly_first_5_categories.txt"
f = open(path, "a+")

while i < len(monthly_ids):
    monthly_id = monthly_ids[i]
    observations_page = observations(series_id=monthly_id,\
                                     realtime_start='2019-11-01', \
                                     realtime_end='2019-11-01')
    ts_values = parse_page(observations_page)
    
    f.write(','.join(ts_values) + '\n')
    
    # to see progress
    i += 1
    if i % 100 == 0:
        print(i)
        
f.close()

In [ ]:
i

In [ ]:
i = 0

path = f"fred_series_5_first_categories/quarterly_first_5_categories.txt"
f = open(path, "a+")

while i < len(quarterly_ids):
    quarterly_id = quarterly_ids[i]
    observations_page = observations(series_id=quarterly_id,\
                                     realtime_start='2019-11-01', \
                                     realtime_end='2019-11-01')
    ts_values = parse_page(observations_page)
    
    f.write(','.join(ts_values) + '\n')
    
    # to see progress
    i += 1
    if i % 100 == 0:
        print(i)
        
f.close()

In [ ]:
i

In [ ]:
i = 0

path = f"fred_series_5_first_categories/anually_first_5_categories.txt"
f = open(path, "a+")

while i < len(annualy_ids):
    annualy_id = annualy_ids[i]
    observations_page = observations(series_id=annualy_id,\
                                     realtime_start='2019-11-01', \
                                     realtime_end='2019-11-01')
    ts_values = parse_page(observations_page)
    
    f.write(','.join(ts_values) + '\n')
    
    # to see progress
    i += 1
    if i % 100 == 0:
        print(i)
        
f.close()

In [ ]:
i

running this script could take about one week to load all series from one IP adress (we used two and done in 5 days)